# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


### 1. What test is appropriate for this problem? Does CLT apply?

In [6]:
len(data)

4870

Since we are comparing 2 different groups, permutation test will be the best test to use. The sample size is greater than 30; therefore, the Central Limit Theorem applies.

### 2. What are the null and alternate hypotheses? <br>

$H_0$: There is no difference in callbacks between resumes with white-sounding names and black-sounding names. <br>
$H_A$: There is a difference in callbacks.

In [11]:
w = data[data.race=='w'].call
b = data[data.race=='b'].call

### 3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [18]:
# Computing statistics for group w
w_mean = np.mean(w)
w_std = np.std(w)

# Computing statistics for group b
b_mean = np.mean(b)
b_std = np.std(b)

# Difference Stats
diff_mean = abs(b_mean - w_mean)
diff_se = np.sqrt(w_std**2/len(w) + b_std**2/len(b))

# Margin of error
moe = 1.96 * diff_se
lower_bd = diff_mean - moe
upper_bd = diff_mean + moe

print(f'Mean Difference: {diff_mean}', 
      f'\nMargin of error: {moe}', 
      f'\n95% Confidence Interval: [{lower_bd}, {upper_bd}]')

Mean Difference: 0.03203285485506058 
Margin of error: 0.015255284385449893 
95% Confidence Interval: [0.016777570469610682, 0.04728813924051047]


**Two Sample Z-Test (Critical Value: 0.01)**

In [21]:
z_stat = (diff_mean - 0)/diff_se
z_pval = stats.norm.sf(abs(z_stat))*2 #twosided
print(f"z statistic: {z_stat}", f"\np-value: {round(z_pval,4)}")

z statistic: 4.115583422082968 
p-value: 0.0


The p-value derived from the 2 sample z-test is extremely small; therefore, the null hypothesis can be rejected.

**Bootstrap Permutation (Critical Value: 0.01)**

In [23]:
bootstrap_replicates = np.empty(10000)

for i in range (10000):
    perm = np.random.permutation(np.concatenate((w,b)))
    w_perm = perm[:len(w)]
    b_perm = perm[len(w):]
    bootstrap_replicates[i] = abs(np.mean(w_perm) - np.mean(b_perm))

perm_pval = np.sum(bootstrap_replicates >= diff_mean)/10000
print(f'p-value: {perm_pval}')

p-value: 0.0


The p-value derived from the bootstrapping permutation test is also extremely small; therefore, the null hypothesis can be rejected. <br>

### 4. Write a story describing the statistical significance in the context or the original problem. <br>
In this experiment, researchers randomly assigned identical résumés to black-sounding or white-sounding names and observed the impact on requests for interviews from employers in the US Labor market. After investigating and applying two separate hypothesis tests, this experiment provides evidence that there is racial discrimination in the US Labor market. <br>

The null hypothesis was rejected in both tests. The p-values for both the two sample z-test and the permutation test are statistically significant since they are well below the 0.01 threshold. This means there's a difference in the number of callbacks between the resumes with white-sounding and black-sounding names.

### 5.  Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis? <br>

No, my analysis does not test that race/name is the most important factor in callback success; it only concludes that race/name is one of the factors. The other columns in the data must also be investigated to determine which factors affect callback success the most.